In [6]:
import pickle
import pandas as pd

In [ ]:
# Dataset building
ticker = "ES"
interval = 30                       # Assumed to be either 5, 10, 15, 30, 60 minutes
df_partition = [0.65, 0.8]          # Training and cross validation set split - remaining is passed to test set

with open("data/futures_ticks.pkl", "rb") as f:
    ticks = pickle.load(f)[ticker]

ticker += "_temp"       # TEMP!!!!

# Pull historical data on defined future ticker
futures_df = pd.read_csv(f"./data/S&P500 Mini/{ticker}.csv", index_col='t', parse_dates=True)

# Resample dataframe to defined holding period
resampled_o = futures_df['o'].resample(f"{interval}min").first()
resampled_h = futures_df['h'].resample(f"{interval}min").max()
resampled_l = futures_df['l'].resample(f"{interval}min").min()
resampled_c = futures_df['c'].resample(f"{interval}min").last()
resampled_v = futures_df['v'].resample(f"{interval}min").sum()

main_df = pd.DataFrame(
    {
        'o': resampled_o,
        'h': resampled_h,
        'l': resampled_l,
        'c': resampled_c,
        'v': resampled_v,
    }
)

# Split into train, CV, and test sets
train_df = main_df[:int(df_partition[0]*len(main_df))]
cv_df = main_df[int(df_partition[0]*len(main_df)):int(df_partition[1]*len(main_df))]
test_df = main_df[int(df_partition[1]*len(main_df)):]

In [ ]:
# main_df["fwd_r_u"] = ((main_df["h"] - main_df["o"]) / ticks).shift(-1)
# main_df["fwd_r_d"] = ((main_df["o"] - main_df["l"]) / ticks).shift(-1)
# main_df["fwd_r_o"] = main_df["fwd_r_u"] + main_df["fwd_r_d"]

In [ ]:
train_df["fwd_r_u"] = ((train_df["h"] - train_df["o"]) / ticks).shift(-1)
train_df["fwd_r_d"] = ((train_df["o"] - train_df["l"]) / ticks).shift(-1)
train_df["fwd_r_o"] = train_df["fwd_r_u"] + train_df["fwd_r_d"]

# Define boundaries, other hyperparameters


# Loop through each row
    # Calculate features: Volume, Volatility, Delta
    # Once calculated, bin ranges into pdf

# Add onto df for reference

,o,h,l,c,v
t,,,,,
2019-12-15 18:00:00,3179.50,3180.50,3177.00,3178.50,5571
2019-12-15 18:30:00,3178.50,3180.00,3178.25,3178.50,2996
2019-12-15 19:00:00,3178.50,3180.75,3178.25,3179.25,3417
2019-12-15 19:30:00,3179.25,3181.00,3178.75,3180.00,2393
2019-12-15 20:00:00,3180.00,3181.25,3179.75,3180.75,3291
...,...,...,...,...,...
2020-04-14 01:00:00,2805.25,2808.00,2796.75,2803.75,7155
2020-04-14 01:30:00,2803.75,2807.75,2794.00,2797.00,6466
2020-04-14 02:00:00,2797.25,2799.75,2788.25,2791.50,11140


In [ ]:
# Visualization: plots on range up, down, overall PDF and CDF

In [ ]:
# Over CV Set, backtest a methodology of setting limit order

Notes:
- Extra: Using a Poisson process to model hit rate to see if limit order slicing is any good (re: Allevaneda-Stoikov paper)
- https://arxiv.org/pdf/2411.04616